In [1]:
import nltk

In [2]:
from nltk.tokenize import TweetTokenizer

In [3]:
from nltk.probability import FreqDist
from nltk.util import ngrams
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

import re
import pandas as pd
import numpy as np


In [4]:
with open('C:\Users\WIN8\Desktop\Machine Learning\NLP_Movie_Review_Proj\NLP_Movie_Review_Proj\yelp_labelled.txt', 'r') as f:
    review_lines = f.readlines()


In [8]:
# review_lines

In [5]:
print(type(review_lines))

<type 'list'>


In [6]:
print(len(review_lines))

1000


In [9]:
# for one_line in review_lines[0:10]:
#     print(one_line)

In [10]:
reviews = []
review_labels = []

In [11]:
for line in review_lines:
    line_list = line.split("\t")
    review = line_list[0]
    review = review.lower()
    reviews.append(review)
    label = line_list[1]
    label = int(label)
    review_labels.append(label)


In [12]:
# print(reviews)
# print(len(reviews))

In [14]:
# print(len(review_labels))
# print(reviews[0])
# print(review_labels[0])

In [16]:
review_string = " ".join(reviews)
# print(review_string)

In [17]:
tknzr = TweetTokenizer()
review_tokens = tknzr.tokenize(review_string)

In [18]:
# print(review_tokens)
# print(len(review_tokens))

In [19]:

#Removing punctuation
punctuation = re.compile(r'[-.?!,":;()|0-9]')
review_tokens2 = []


In [20]:
for token in review_tokens:
    word = punctuation.sub("", token)
    if len(word)>0:
        review_tokens2.append(word)
        
print(len(review_tokens2))


10848


In [21]:
#Removing stopwords
# nltk.download('stopwords')
# from nltk.corpus import stopwords
#  set(stopwords.words('english'))
review_tokens3 = []
stp_words = set(stopwords.words('english'))


In [22]:
for token in review_tokens2: # remove stop words
    token = token.lower()
    if token not in stp_words:
        review_tokens3.append(token)
        
print(len(review_tokens3))


5588


In [25]:
#Finding Frequency Distribution

fdist = FreqDist() #dictionary with keys as word and values as count
for word in review_tokens3:
    fdist[word]+=1


In [26]:
# fdist
# print(fdist)

In [27]:
# fdist_top20 = fdist.most_common(20) #top 20 tokens with highest frequency

In [28]:
# fdist_top20

In [29]:
nltk.download('averaged_perceptron_tagger')
pos_list = [] # POS=Part Of Speech
for token in review_tokens3: # List after stopwords and punctuation
    pos_list.append(nltk.pos_tag([token]))# The word should be inside [], else tagger will take it as string


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\WIN8\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [32]:
# print(len(pos_list))

print(pos_list[0])

pos_set = set()


[(u'wow', 'NN')]


In [33]:

for pos in pos_list: # Finding all different Part of Speech Tags
    pos_set.add(pos[0][1])
    
print(len(pos_set))

print(pos_set)


20
set(['MD', 'VB', 'VBG', 'RB', 'NN', 'VBD', 'CC', 'JJS', 'VBN', 'RBR', "''", 'PRP', 'JJ', 'IN', 'VBZ', 'DT', 'CD', 'NNS', 'JJR', '$'])


In [36]:
#Finding Adjectives
pos_JJ = []

for each_POS in pos_list:
    if each_POS[0][1] in ["JJ","JJR","JJS"]:
        pos_JJ.append(each_POS[0][0])

# print(pos_JJ)

In [37]:
fdist_JJ = FreqDist()
for word in pos_JJ:
    fdist_JJ[word] += 1
    
print(len(fdist_JJ))


166


In [38]:
fdist_JJ_top20 = fdist_JJ.most_common(20) #top 20 tokens with highest frequencies

print(fdist_JJ_top20)


[(u'good', 95), (u'great', 70), (u'best', 30), (u'nice', 25), (u'delicious', 23), (u'disappointed', 19), (u'bad', 18), (u'much', 18), (u'worst', 15), (u'fresh', 14), (u'fantastic', 12), (u'next', 11), (u'terrible', 10), (u'little', 10), (u'many', 9), (u'hot', 9), (u'overall', 8), (u'impressed', 8), (u'happy', 8), (u'attentive', 7)]


In [40]:
#Lemmatizing (eg: -> rocks : rock-> corpora : corpus-> better : good) words

word_lem = WordNetLemmatizer()
lem_ADJ = []
lem_ADV = []
lem_VERB = []


In [41]:
nltk.download('wordnet')
for word in review_tokens3:
    word_pos =nltk.pos_tag([word])
    
    if word_pos[0][1] in ["JJ", "JJR", "JJS"]:
        lem_ADJ.append((word_pos[0][0], word_lem.lemmatize(word, wordnet.ADJ)))
        
    if word_pos[0][1] in ["RB","RBR","RBS"]:
        lem_ADV.append((word_pos[0][0], word_lem.lemmatize(word, wordnet.VERB)))
        
    if word_pos[0][1] in ["VB","VBD","VBG","VBN","VBZ"]:
        lem_VERB.append((word_pos[0][0], word_lem.lemmatize(word, wordnet.VERB)))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WIN8\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:

# print(len(lem_ADJ))    


In [99]:
# print(lem_ADJ[:10])

In [100]:
# print(lem_ADV)

In [101]:
# print(lem_ADV[:10])

# print(lem_VERB)

# print(lem_VERB[:10])


In [43]:
# Checking most frequent n-grams

#Segregating all positive and negative reviews

positive_reviews = []
negative_reviews = []

for line in review_lines: # Splitting reviews into text and labels
    line_list = line.split("\t")
    
    label = line_list[1]
    label = int(label)
    
    if label==1:
        pos_review = line_list[0]
        pos_review = pos_review.lower()
        positive_reviews.append(pos_review)
        
    elif label==0:
        neg_review = line_list[0]
        neg_review = neg_review.lower()
        negative_reviews.append(neg_review)
        
print(len(positive_reviews))
print(len(negative_reviews))


500
500


In [44]:
#Working on positive reviews
nltk.download('punkt')

pos_review_string = " ".join(positive_reviews)

pos_rev_tokens = nltk.word_tokenize(pos_review_string)

pos_rev_trigrams = list(nltk.trigrams(pos_rev_tokens))

print(len(pos_rev_trigrams))

print(pos_rev_trigrams[0])

fdist_pos_rev = nltk.FreqDist(pos_rev_trigrams)

print(fdist_pos_rev.most_common(10))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WIN8\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


6016
('wow', '...', 'loved')
[(('!', '!', '!'), 46), (('this', 'place', 'is'), 12), (('the', 'food', 'was'), 11), (('this', 'place', '.'), 9), (('.', 'the', 'food'), 7), (('.', 'i', 'love'), 7), (('our', 'server', 'was'), 7), (('.', 'this', 'place'), 7), (('and', 'it', 'was'), 6), (('.', 'i', 'had'), 6)]


In [46]:
#Working on Negative Reviews
    
# neg_review_string = " ".join(negative_reviews)

# neg_rev_tokens = nltk.word_tokenize(neg_review_string)

# neg_rev_trigrams = list(nltk.trigrams(neg_rev_tokens))

# print(len(neg_rev_trigrams))

# print(neg_rev_trigrams[0])

# fdist_neg_rev = nltk.FreqDist(neg_rev_trigrams)

# print(fdist_neg_rev.most_common(10))    


In [51]:
#Predicting reviews using machine learning

#Creating features and labels

review_list = []

for line in review_lines: # Splitting reviews into text and labels
    line_list = line.split("\t")
    review = line_list[0]
    review = review.lower()
    review_list.append(review)

# print(len(review_list))

# print(review_list[0][1])


In [52]:
tf_vect = TfidfVectorizer(min_df = 2, lowercase = True, stop_words = 'english')

X_TFIDF = tf_vect.fit_transform(review_list)


In [56]:
# print(type(X_TFIDF))

# print(X_TFIDF.shape)


In [57]:
X_TFIDF_names = tf_vect.get_feature_names()

X_tf = pd.DataFrame(X_TFIDF.toarray(), columns = X_TFIDF_names)


In [58]:
# print(X_tf.head())


In [60]:
y = pd.Series(review_labels)

# print(type(y))

# print(y.shape)

# print(y.head())


In [64]:
#Train-test Split

X_TF_train, X_TF_test, y_TF_train, y_TF_test = train_test_split(X_tf, y, test_size=0.2, random_state=5)


In [61]:
'''
print(X_TF_train.shape)
print(X_TF_test.shape)
print(y_TF_train.shape)
print(y_TF_test.shape)
'''

'\nprint(X_TF_train.shape)\nprint(X_TF_test.shape)\nprint(y_TF_train.shape)\nprint(y_TF_test.shape)\n'

In [65]:
#Using Multinomial Naive Bayes
clf_TF = MultinomialNB()
clf_TF.fit(X_TF_train, y_TF_train)

y_TF_pred = clf_TF.predict(X_TF_test)


In [62]:
# print(type(y_TF_pred))

# print(y_TF_pred.shape)

# print(metrics.accuracy_score(y_TF_test, y_TF_pred))


In [68]:
# score_clf_TF = confusion_matrix(y_TF_test, y_TF_pred)


In [69]:
# print(score_clf_TF)


In [70]:
# TP = score_clf_TF[0][0]
# FP = score_clf_TF[0][1]
# FN = score_clf_TF[1][0]
# TN = score_clf_TF[1][1]


In [71]:
# print('True Positive: ',TP)
# print('False Positive: ', FP)
# print('False Negative: ', FN)
# print('True Negative: ', TN)


In [72]:
# print("Correctly Identified: ", TP+TN)
# print("Wrongly Identified: ", FP+FN)


In [73]:
#Checking Classifier on new Data

def find_sentiments (sentence):
    sent_list = [sentence]
    sent_vect = tf_vect.transform(sent_list)
    sent_pred = clf_TF.predict(sent_vect)
    print("Sentiment: ",sent_pred[0])
    return


In [74]:

find_sentiments("I liked the food a lot")
find_sentiments("The food was very bad")
find_sentiments("The place was good but the food was bad")

('Sentiment: ', 1)
('Sentiment: ', 0)
('Sentiment: ', 0)
